In [ ]:
import pandas as pd
import numpy as np
import glob
import gzip
import shutil
import os
from tqdm import tqdm

# set infinite display
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

## Merging the two metadata tables. Many case_ids are duplicated, but these are tied to the presence of multiple read_group_ids and so the duplicates are not filtered out at this stage.

In [ ]:
# convert metadata tsvs into dataframes
df = pd.read_csv('../../data/CPTAC-3.bio_repl_query.sur.tsv', sep='\t', header=0)
df2 = pd.read_csv('../../data/CPTAC-3.gdc_repl_query.tsv', sep='\t', header=0)

In [ ]:
# get shape of the dataframe
print(df.shape)
# get shape of the dataframe
print(df2.shape)

In [ ]:
# check duplicated case_id in df
print(df.case_id.duplicated().sum())
# check duplicated case_id in df2
print(df2.case_id.duplicated().sum())

In [ ]:
# join both by case_id and keep all rows
df3 = df.merge(df2, how='outer', on='case_id', suffixes=('_bio', '_gdc'))

# get shape of the dataframe
df3.shape

In [ ]:
# remove the column read_pair_number and submitted_unaligned_reads_id
df3 = df3.drop(['read_pair_number'], axis=1)
df3 = df3.drop(['submitted_unaligned_reads_id'], axis=1)

# drop duplicated and save as new dataframe
df4 = df3.drop_duplicates()

# print shape of the old and new dataframes
print(df3.shape)
print(df4.shape)

In [ ]:
# check duplicate case_id
df4[df4.duplicated(['case_id'], keep=False)].shape

### A few columns were found in both metadata tables and the following chunk of code makes sure values from both are kept in a single column.

In [ ]:
# Iterate over each pair of columns that were repeated in both initial dataframes and combine the values
for col in df4.columns:
    if col.endswith('_bio'):
        gdc_col = col.replace('_bio', '_gdc')
        if gdc_col in df4.columns:
            new_col_name = col.replace('_bio', '')  # Create a new column name without suffixes
            new_column_values = []
            # Iterate down the rows in the DataFrame
            for i in range(len(df4)):
                bio_value = df4[col].iloc[i]
                gdc_value = df4[gdc_col].iloc[i]
                if not pd.isnull(bio_value):
                    new_column_values.append(bio_value)
                else:
                    new_column_values.append(gdc_value)
            df4[new_col_name] = new_column_values  # Add a new column with combined values
            df4.drop([col, gdc_col], axis=1, inplace=True)  # Delete the original column pair

## The following code starts from the unzipped contents of the .tgz file we got from box, unzips the subfiles if they haven't been unzipped already, and then compiles the contents and filenames into a TPM dataframe.

In [ ]:
source_dir = "../../data/local_results/"  # Change this to the source directory where your .sf.gz files are located
destination_dir = "../../data/local_results/sf"

# Get a list of all files in the source directory
filenames = os.listdir(source_dir)

for filename in filenames:
    if filename.endswith('.sf.gz'):
        source_path = os.path.join(source_dir, filename)
        destination_path = os.path.join(destination_dir, filename[:-3])

        # Check if the destination file already exists
        if not os.path.exists(destination_path):
            with gzip.open(source_path, 'rb') as f_in:
                with open(destination_path, 'wb') as f_out:
                    shutil.copyfileobj(f_in, f_out)
        else:
            print(f"Skipping {filename} as it has already been unzipped.")

In [ ]:
# Define the directory where the '.sf' files are located
sf_directory = "../../data/local_results/sf/"

# Initialize an empty DataFrame to store the results
result_df = pd.DataFrame()

# List the '.sf' files in the directory
sf_files = [f for f in os.listdir(sf_directory) if f.endswith('.sf')]

# loop through all .sf files
for file in tqdm(sf_files):
    # read in the file
    df = pd.read_csv("../../data/local_results/sf/" + file, sep='\t', header=0)
    # keep only the Name and TPM columns
    df = df[['Name', 'TPM']]
    # rename the TPM column to the filename without the .sf extension
    df = df.rename(columns={'TPM': file.split('.')[0]})
    # if results_df is empty, then set it to the df dataframe
    if result_df.empty:
        result_df = df
    else:
        # merge the dataframe with the result_df dataframe
        result_df = pd.merge(result_df, df, on='Name', how='outer')
    del df

### The dataframes are saved as .csv files

In [ ]:
# save the results_df dataframe as a tsv file with the name TPM.tsv
result_df.to_csv('../../results/TPM.tsv', sep='\t', index=False)
# save the metadata dataframe
df4.to_csv('../../results/metadata.tsv', sep='\t', index=False)